In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
train_url = '/home/thinker/NLP/NLP/FromScratch/error detection/data/data/train_data.csv'
test_url = '/home/thinker/NLP/NLP/FromScratch/error detection/data/data/test_data.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/home/thinker/NLP/NLP/FromScratch/error detection/data/stopwords_bangla.xlsx',index_col=False)

In [2]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [3]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

In [4]:
df_test

,Video Title,Genre,Comment,Error,Category,Correct form
0,'নেত্রীর কথা শুনলে এখন হাজার হাজার শ্রমিক রাস্...,Politics,আওয়ামী লীগের এতো লোক ভয় পায়,0,NaN,NaN
1,RedMagic 8 Pro দেখে আমি তো অবাক 😮,Miscellaneous,প্রাইস বললে হতো রে ইমন,1,Code Switching,দাম টা বললে কি হতো রে ইমন
2,অস্থির বাঙালি Part 35😂 osthir bengali | funny ...,Entertainment,সুন্দর হাসি ভালো লাগল,1,Spelling,এত সুন্দর হাসি ভালো লাগলো
3,দেশে প্রথমবারের মতো চ্যানেল 24-এর পর্দায় সংবাদ...,News,সময় এসেছে আপনাদেরকে বিদায় জানাবার,0,NaN,NaN
4,"মীনা, রাজু, মিঠুর মিমিক্রি করে তাক লাগিয়ে দিয়ে...",Entertainment,চুল একটু বড় ভালো না,1,Spelling,চুল একটু বড় হলে ভালো হতো না
...,...,...,...,...,...,...
2006,ভাইভা দিতে গিয়ে উল্টো ভাইভা নিলেন মোশাররফ,Entertainment,এইরকম জীবনেও চাকরী না,0,NaN,NaN
2007,জিয়াউর রহমান: বিএনপির প্রতিষ্ঠাতা ও সাবেক রাষ...,Politics,ঘটনায় আপনাদের সাংবাদিক কুমিল্লায় তথ্য যাইনি,0,NaN,NaN
2008,খালেদা জিয়া’র ব্যাপারে আর কত করবো ? প্রশ্ন প্র...,Politics,করব মরলনা,1,Spelling,আর কত করবো তাও তো মরলো না
2009,হাবলংগের বাজারে | Hablonger Bazare | Humayun A...,Entertainment,হুমায়ূন আহমেদের তৈরি সোনার কাল্পনিক বাংলাদেশ ...,0,NaN,NaN


In [5]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(df_train.Error)
Test_Y = Encoder.fit_transform(df_test.Error)

In [6]:
df_all  = pd.concat([df_train, df_test], ignore_index=True)
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df_all['Comment'])
Train_X_Tfidf = Tfidf_vect.transform(df_train['Comment'])
Test_X_Tfidf = Tfidf_vect.transform(df_test['Comment'])

In [7]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.6307    0.8663    0.7300      1167
           1     0.6176    0.2986    0.4026       844

    accuracy                         0.6280      2011
   macro avg     0.6242    0.5825    0.5663      2011
weighted avg     0.6252    0.6280    0.5926      2011

